In [2]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import keras as K
import keras_tuner as kt

In [6]:
def build_model(hp):
    model = K.Sequential()

    # First Convolutional Block
    model.add(K.layers.Conv2D(
        filters=hp.Int('conv_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv_kernel', values=[3, 5]),
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(K.layers.MaxPooling2D(pool_size=(2, 2)))

    # Second Convolutional Block
    model.add(K.layers.Conv2D(
        filters=hp.Int('conv_filters_2', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv_kernel_2', values=[3, 5]),
        activation='relu'
    ))
    model.add(K.layers.MaxPooling2D(pool_size=(2, 2)))

    # Fully Connected Layers
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(
        units=hp.Int('dense_units', min_value=64, max_value=256, step=64),
        activation='relu'
    ))
    model.add(K.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(K.layers.Dense(1, activation='sigmoid'))  # Binary classification

    # Compile with tunable learning rate
    model.compile(
        optimizer=K.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [7]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Increase this for a broader search
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='cifar10_tuning'
)



In [8]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

# Note: y_train and y_test are of shape (num_samples, 1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)


In [ ]:
tuner.search(
    x_train, y_train,
    epochs=5,
    validation_data=(x_test, y_test)
)


Trial 1 Complete [00h 10m 31s]
val_accuracy: 0.10000000149011612

Best val_accuracy So Far: 0.10000000149011612
Total elapsed time: 00h 10m 31s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
32                |32                |conv_filters
3                 |3                 |conv_kernel
64                |128               |conv_filters_2
5                 |3                 |conv_kernel_2
128               |192               |dense_units
0.3               |0.2               |dropout_rate
0.0037554         |0.00064214        |learning_rate

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 98s 62ms/step - accuracy: 0.0981 - loss: -2304569769984.0000 - val_accuracy: 0.1000 - val_loss: -53522393464832.0000
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 140s 60ms/step - accuracy: 0.0999 - loss: -134735581937664.0000 - val_accuracy: 0.1000 - val_loss: -591450014220288.0000
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 141s 60ms/step - accuracy: 0.1008 - loss: -898043368964

In [ ]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters found:")
print(f"Conv Filters: {best_hp.get('conv_filters')}")
print(f"Conv Kernel: {best_hp.get('conv_kernel')}")
print(f"Conv Filters 2: {best_hp.get('conv_filters_2')}")
print(f"Conv Kernel 2: {best_hp.get('conv_kernel_2')}")
print(f"Dense Units: {best_hp.get('dense_units')}")
print(f"Dropout Rate: {best_hp.get('dropout_rate')}")
print(f"Learning Rate: {best_hp.get('learning_rate')}")
